In [1]:
import pandas as pd
import itertools

In [2]:
data=pd.read_csv('test_stripped.csv')

In [3]:
groups=data[['skill_category', 'no_stopwords_stemmed']]

In [4]:
d_group=groups.groupby('skill_category')['no_stopwords_stemmed'].apply(' '.join).reset_index()

In [5]:
frequency_dict=[dict(pd.Series(i.split()).value_counts()) for i in d_group.no_stopwords_stemmed]

In [6]:
top_profile_words={}
for i,k in zip(d_group.skill_category,frequency_dict):
    top_profile_words[i]=[p for p,s in k.items() if s>500]

In [34]:
%%time
ids=[]
skill_ranks=[]
skills=[]
for j,i in enumerate(data.no_stopwords_stemmed):
    for k in top_profile_words:
        ids.append(j)
        skills.append(k)
        skill_ranks.append((len(list(set([p for p in i.split() if p in top_profile_words[k]])))/len(top_profile_words[k]))*100)

CPU times: user 2min 12s, sys: 24 ms, total: 2min 13s
Wall time: 2min 13s


In [35]:
df=pd.concat([pd.Series(ids, name='id'),pd.Series(skills, name='skill_category'), 
              pd.Series(skill_ranks, name='skill_category_alignment')], axis=1)

df['skill_category']=[i.replace('.xls','') for i in df['skill_category']]

In [36]:
%%time
df_sorted=[]
for i in range(0,len(df.id.unique())):
    df_sorted.append(df[df.id==i].sort_values(['skill_category_alignment'], ascending=False))

CPU times: user 18.8 s, sys: 36 ms, total: 18.8 s
Wall time: 18.8 s


In [42]:
%%time
sorted_ids=[]
sorted_skill_categories=[]
sorted_rank=[]
sorted_score=[]

for i in df_sorted:
    i=i.reset_index()
    sorted_ids.append(['ID_'+str(p) for p in i.id])
    sorted_skill_categories.append(i.skill_category)
    sorted_rank.append(list(range(1,len(i)+1)))
    sorted_score.append(i.skill_category_alignment)

CPU times: user 8.59 s, sys: 28 ms, total: 8.62 s
Wall time: 8.62 s


In [43]:
%%time
df_full=pd.concat([pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_ids])), name='ids'),
                   pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_skill_categories])), 
                             name='skill_category'),
                  pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_rank])), 
                            name='rank'),
                  pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_score])), 
                            name='score')], axis=1)

CPU times: user 530 ms, sys: 24 ms, total: 554 ms
Wall time: 553 ms


In [44]:
df_full

,ids,skill_category,rank,score
0,ID_0,Science,1,70.000000
1,ID_0,Speaking,2,68.888889
2,ID_0,Instructing,3,68.852459
3,ID_0,Social_Perceptiveness,4,68.656716
4,ID_0,Equipment_Maintenance,5,68.627451
...,...,...,...,...
766117,ID_22532,Coordination,30,61.971831
766118,ID_22532,Management_of_Material_Resources,31,61.875000
766119,ID_22532,Time_Management,32,60.645161
766120,ID_22532,Service_Orientation,33,60.583942


In [46]:
#df_full.to_csv('df_full.csv')

In [47]:
df_full10000=df_full.iloc[:10000,:]
#df_full10000.to_csv('df_full_10000.csv')